In [ ]:

# 画像データを用いてVGG19モデルをベースとした転移学習を行う

# 必要なライブラリのインポート
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.optimizers import RMSprop
from keras.layers import Input, Flatten, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

# 画像の読み込みと前処理の関数
def load_and_preprocess_images(directory, target_size=(300, 300), label=0):
    x, y = [], []
    for filename in os.listdir(directory):
        if filename.endswith('.png'):  # PNG画像のみを対象
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=target_size, color_mode='rgb')
            img_array = img_to_array(img) / 255.0  # 画像を配列に変換し、正規化
            x.append(img_array)
            y.append(label)
    return np.array(x), np_utils.to_categorical(y, num_classes=2)

# モデルの構築
def build_model(input_shape=(300, 300, 3)):
    input_tensor = Input(shape=input_shape)
    base_model = VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)
    x = Flatten()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# 訓練と評価の実行
def train_and_evaluate(x_train, y_train, x_val, y_val, file_name, epochs=50, batch_size=16):
    model = build_model()
    os.makedirs(file_name, exist_ok=True)
    checkpoint_path = os.path.join(file_name, 'model_{epoch:02d}.h5')
    checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6, verbose=1)

    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,
                        validation_data=(x_val, y_val), callbacks=[checkpoint, reduce_lr])

    # 訓練履歴を保存
    pd.DataFrame(history.history).to_csv(os.path.join(file_name, 'acc_loss.csv'))

    # 損失と精度のグラフを保存
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(file_name, 'training_validation_metrics.png'))
    plt.close()

# データの読み込み
path1_under = "/path/to/train/under"
path1_over = "/path/to/train/over"
path2_under = "/path/to/validation/under"
path2_over = "/path/to/validation/over"

x_train_under, y_train_under = load_and_preprocess_images(path1_under, label=0)
x_train_over, y_train_over = load_and_preprocess_images(path1_over, label=1)
x_train = np.concatenate([x_train_under, x_train_over])
y_train = np.concatenate([y_train_under, y_train_over])

x_val_under, y_val_under = load_and_preprocess_images(path2_under, label=0)
x_val_over, y_val_over = load_and_preprocess_images(path2_over, label=1)
x_val = np.concatenate([x_val_under, x_val_over])
y_val = np.concatenate([y_val_under, y_val_over])

# 訓練の実行
train_and_evaluate(x_train, y_train, x_val, y_val, 'model_output')


















